In [16]:
import pandas as pd

data = pd.read_csv("processed_data.csv")
data.head()

,Text,Labels
0,subject naturally irresistible corporate ident...,1
1,subject stock trading gunslinger fanny merrill...,1
2,subject unbelievable new homes made easy im wa...,1
3,subject 4 color printing special request addit...,1
4,subject money get software cds software compat...,1


In [17]:
import numpy as np
import re
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Saniya and
[nltk_data]     Family\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Saniya and
[nltk_data]     Family\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [121]:
X = data["Text"]
y = data["Labels"]

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

documents

['subject naturally irresistible corporate identity lt really hard recollect company market full suqgestions information isoverwhelminq good catchy logo stylish statlonery outstanding website make task much easier promise havinq ordered iogo company automaticaily become world ieader isguite ciear without good product effective business organization practicable aim hotat nowadays market promise marketing effort become much effective list clear benefit creativeness hand made original logo specially done reflect distinctive company image convenience logo stationery provided format easy use content management system letsyou change website content even structure promptness see logo draft within three business day affordability marketing break make gap budget 100 satisfaction guaranteed provide unlimited amount change extra fee surethat love result collaboration look portfolio interested',
 'subject stock trading gunslinger fanny merrill muzo colza attainder penultimate like esmark perspicuo

In [122]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=15, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()
X

array([[0, 0, 2, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [124]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
X

array([[0.        , 0.        , 0.50492764, ..., 0.20523625, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [126]:
from sklearn.ensemble import RandomForestClassifier 

classifier = RandomForestClassifier(n_estimators=10, random_state=0)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [127]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[2083  111]
 [ 201 1137]]
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      2194
           1       0.91      0.85      0.88      1338

    accuracy                           0.91      3532
   macro avg       0.91      0.90      0.90      3532
weighted avg       0.91      0.91      0.91      3532

0.9116647791619479


In [152]:
def predict(i):
    text = []
    document = re.sub(r'\W', ' ', str(i))

    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    document = re.sub(r'^b\s+', '', document)

    document = document.lower()

    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    text.append(document)
    
    if len(text[0]) < 15:
        return(222)
        
    else:
        vectorizer = CountVectorizer(max_features=15, min_df=0.7, max_df=1, stop_words=stopwords.words('english'))

        x = vectorizer.fit_transform(text).toarray()
        x = tfidfconverter.fit_transform(x).toarray()
        x.resize((1, 15), refcheck=False)
        t = x.reshape(1,-1)
        y = classifier.predict(t)
        return(y[0])

In [153]:
i = input("Enter mail:")
predict(i)

0